In [1]:
import os,sys
import torch
import torchvision
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import torchvision.models.resnet as models 
import torch.nn.functional as F  
from PIL import Image
import h5py
import numpy as np
import scipy.io as sio
import pickle
import pdb
import matplotlib.pyplot as plt
import pandas as pd
import gensim.downloader as api
import torch.optim as optim
import importlib

from DAZLE_Sushree import DAZLE
from CUBDataLoader import CUBDataLoader
from helper_func import eval_zs_gzsl,visualize_attention#,get_attribute_attention_stats

In [2]:
data_path = 'C:/Sushree/Jio_Institute/Dataset/'
feature_path = 'C:/Sushree/Jio_Institute/Dataset/CUB_200_2011/CUB_200_2011/'

dataloader = CUBDataLoader(data_path, feature_path, device = None, is_balance=False)

C:/Sushree/Jio_Institute/Dataset/
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
CUB
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
_____
C:/Sushree/Jio_Institute/Dataset/CUB_200_2011/CUB_200_2011/feature_map_ResNet_101_CUB.hdf5
Expert Attr


In [3]:
dataloader.augment_img_path()

def get_lr(optimizer):
    lr = []
    for param_group in optimizer.param_groups:
        lr.append(param_group['lr'])
    return lr

In [4]:
seed = 200
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)

# define inputs to the network
batch_size = 32
nepoches = 100
niters = dataloader.ntrain * nepoches//batch_size # 22053 for AWA2
print('niters', niters)

dim_f = 2048
dim_v = 300
init_w2v_att = dataloader.w2v_att # load the attribute semantic vectors
att = dataloader.att
normalize_att = dataloader.normalize_att


trainable_w2v = True
lambda_ = 0.1
#normalize_V=True
#normalize_F=True
#is_conservative=True
bias = 0
prob_prune = 0
uniform_att_1 = False
uniform_att_2 = False


#is_conv=False
#is_bias=True
#non_linear_act=False
#loss_type = 'CE'
#non_linear_emb = False
#is_sigmoid = False

seenclass = dataloader.seenclasses #load seen and unseen data
unseenclass = dataloader.unseenclasses 
desired_mass = 1
report_interval = niters//nepoches

device = None

model = DAZLE(dim_f,dim_v,init_w2v_att,att,normalize_att,
            seenclass,unseenclass,
            lambda_,
            trainable_w2v,normalize_V=False,normalize_F=True,is_conservative=True,
            uniform_att_1=uniform_att_1,uniform_att_2=uniform_att_2,
            prob_prune=prob_prune,desired_mass=desired_mass, is_conv=False,
            is_bias=True)
model.to(device)

setup = {'pmp':{'init_lambda':0.1,'final_lambda':0.1,'phase':0.8},
         'desired_mass':{'init_lambda':-1,'final_lambda':-1,'phase':0.8}}
print(setup)

params_to_update = []
params_names = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        params_names.append(name)
        print("\t",name)
#%%
lr = 0.0001
weight_decay = 0.0001
momentum = 0.9
#%%
lr_seperator = 1
lr_factor = 1
print('default lr {} {}x lr {}'.format(params_names[:lr_seperator],lr_factor,params_names[lr_seperator:]))
optimizer  = optim.RMSprop( params_to_update ,lr=lr,weight_decay=weight_decay, momentum=momentum)
print('-'*30)
print('learing rate {}'.format(lr))
print('trainable V {}'.format(trainable_w2v))
print('lambda_ {}'.format(lambda_))
print('optimized seen only')
print('optimizer: RMSProp with momentum = {} and weight_decay = {}'.format(momentum,weight_decay))
print('-'*30)

niters 22053
------------------------------
Configuration
loss_type CE
normalize_V = False: no constraint V
normalize_F = True: normalize F
is_conservative = True: training to exclude unseen class [seen upperbound]
Init word2vec
non_linear_act = False: Linear model
loss_att BCEWithLogitsLoss()
Bilinear attention module
******************************
Measure w2v deviation
Compute Pruning loss Parameter containing:
tensor(0)
is_bias = True: Add one smoothing
Second layer attenion conditioned on image features
------------------------------
is_sigmoid = False: No sigmoid on attr score
{'pmp': {'init_lambda': 0.1, 'final_lambda': 0.1, 'phase': 0.8}, 'desired_mass': {'init_lambda': -1, 'final_lambda': -1, 'phase': 0.8}}
	 V
	 W_1
	 W_2
	 W_3
default lr ['V'] 1x lr ['W_1', 'W_2', 'W_3']
------------------------------
learing rate 0.0001
trainable V True
lambda_ 0.1
optimized seen only
optimizer: RMSProp with momentum = 0.9 and weight_decay = 0.0001
------------------------------


C:\Users\Sushree.Behera\Sushree\GZSL_Implementation\DAZLE_Sushree.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.init_w2v_att = F.normalize(torch.tensor(init_w2v_att))
C:\Users\Sushree.Behera\Sushree\GZSL_Implementation\DAZLE_Sushree.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.att = nn.Parameter(F.normalize(torch.tensor(att)),requires_grad = False)


In [5]:
best_performance = [0,0,0,0]
for i in range(0,niters):
    model.train()
    optimizer.zero_grad()
    
    cal_epoc = np.ceil((batch_size * i)/dataloader.ntrain)
    
    batch_label, batch_feature, batch_att = dataloader.next_batch(batch_size)
    out_package = model(batch_feature)
    
    in_package = out_package
    in_package['batch_label'] = batch_label
    
    out_package=model.compute_loss(in_package)
    loss, loss_CE, loss_cal = out_package['loss'], out_package['loss_CE'], out_package['loss_cal']
    
    loss.backward()
    optimizer.step()
    if i%report_interval==0:
        print('-'*30)
        # evaluate the model for every report interval
        acc_seen, acc_novel, H, acc_zs = eval_zs_gzsl(dataloader, model, device, bias_seen=-bias, bias_unseen=bias)
        
        if H > best_performance[2]:
            best_performance = [acc_seen, acc_novel, H, acc_zs]
        stats_package = {'epoch': cal_epoc, 'iter':i, 'loss':loss.item(), 'loss_CE':loss_CE.item(),
                         'loss_cal': loss_cal.item(),
                         'acc_seen':best_performance[0], 'acc_novel':best_performance[1], 'H':best_performance[2], 'acc_zs':best_performance[3]}
        
        print(stats_package)

------------------------------
bias_seen 0 bias_unseen 0
{'epoch': 0.0, 'iter': 0, 'loss': 5.410717964172363, 'loss_CE': 5.377955913543701, 'loss_cal': 0.3276199400424957, 'acc_seen': 0, 'acc_novel': 0, 'H': 0, 'acc_zs': 0}
------------------------------
bias_seen 0 bias_unseen 0
{'epoch': 1.0, 'iter': 220, 'loss': 1.9489353895187378, 'loss_CE': 1.8605239391326904, 'loss_cal': 0.8841143250465393, 'acc_seen': 0.35459935665130615, 'acc_novel': 0.47586262226104736, 'H': 0.40637761629778285, 'acc_zs': 0.5310006141662598}
------------------------------
bias_seen 0 bias_unseen 0
{'epoch': 2.0, 'iter': 440, 'loss': 1.1371495723724365, 'loss_CE': 1.0205491781234741, 'loss_cal': 1.166003942489624, 'acc_seen': 0.4595080018043518, 'acc_novel': 0.5204228758811951, 'H': 0.48807213087162266, 'acc_zs': 0.5963373780250549}
------------------------------
bias_seen 0 bias_unseen 0
{'epoch': 3.0, 'iter': 660, 'loss': 1.042487382888794, 'loss_CE': 0.9094179272651672, 'loss_cal': 1.3306939601898193, 'acc_s

------------------------------
bias_seen 0 bias_unseen 0
{'epoch': 28.0, 'iter': 6160, 'loss': 0.49098071455955505, 'loss_CE': 0.32870346307754517, 'loss_cal': 1.622772455215454, 'acc_seen': 0.5973256826400757, 'acc_novel': 0.5632122159004211, 'H': 0.5797675745997353, 'acc_zs': 0.653940737247467}
------------------------------
bias_seen 0 bias_unseen 0
{'epoch': 29.0, 'iter': 6380, 'loss': 0.5845341086387634, 'loss_CE': 0.44059813022613525, 'loss_cal': 1.4393597841262817, 'acc_seen': 0.5973256826400757, 'acc_novel': 0.5632122159004211, 'H': 0.5797675745997353, 'acc_zs': 0.653940737247467}
------------------------------
bias_seen 0 bias_unseen 0
{'epoch': 30.0, 'iter': 6600, 'loss': 0.5506748557090759, 'loss_CE': 0.4017302691936493, 'loss_cal': 1.489445686340332, 'acc_seen': 0.5973256826400757, 'acc_novel': 0.5632122159004211, 'H': 0.5797675745997353, 'acc_zs': 0.653940737247467}
------------------------------
bias_seen 0 bias_unseen 0
{'epoch': 31.0, 'iter': 6820, 'loss': 0.59632855653

------------------------------
bias_seen 0 bias_unseen 0
{'epoch': 56.0, 'iter': 12320, 'loss': 0.3739486336708069, 'loss_CE': 0.19899369776248932, 'loss_cal': 1.749549388885498, 'acc_seen': 0.5973256826400757, 'acc_novel': 0.5632122159004211, 'H': 0.5797675745997353, 'acc_zs': 0.653940737247467}
------------------------------
bias_seen 0 bias_unseen 0
{'epoch': 57.0, 'iter': 12540, 'loss': 0.4676172137260437, 'loss_CE': 0.33651643991470337, 'loss_cal': 1.3110076189041138, 'acc_seen': 0.5973256826400757, 'acc_novel': 0.5632122159004211, 'H': 0.5797675745997353, 'acc_zs': 0.653940737247467}
------------------------------
bias_seen 0 bias_unseen 0
{'epoch': 58.0, 'iter': 12760, 'loss': 0.4397614002227783, 'loss_CE': 0.2747178077697754, 'loss_cal': 1.6504360437393188, 'acc_seen': 0.5973256826400757, 'acc_novel': 0.5632122159004211, 'H': 0.5797675745997353, 'acc_zs': 0.653940737247467}
------------------------------
bias_seen 0 bias_unseen 0
{'epoch': 59.0, 'iter': 12980, 'loss': 0.3534612

------------------------------
bias_seen 0 bias_unseen 0
{'epoch': 84.0, 'iter': 18480, 'loss': 0.35490405559539795, 'loss_CE': 0.14733366668224335, 'loss_cal': 2.0757038593292236, 'acc_seen': 0.5973256826400757, 'acc_novel': 0.5632122159004211, 'H': 0.5797675745997353, 'acc_zs': 0.653940737247467}
------------------------------
bias_seen 0 bias_unseen 0
{'epoch': 85.0, 'iter': 18700, 'loss': 0.29354196786880493, 'loss_CE': 0.09516231715679169, 'loss_cal': 1.98379647731781, 'acc_seen': 0.5973256826400757, 'acc_novel': 0.5632122159004211, 'H': 0.5797675745997353, 'acc_zs': 0.653940737247467}
------------------------------
bias_seen 0 bias_unseen 0
{'epoch': 86.0, 'iter': 18920, 'loss': 0.37095606327056885, 'loss_CE': 0.22052133083343506, 'loss_cal': 1.5043472051620483, 'acc_seen': 0.5973256826400757, 'acc_novel': 0.5632122159004211, 'H': 0.5797675745997353, 'acc_zs': 0.653940737247467}
------------------------------
bias_seen 0 bias_unseen 0
{'epoch': 87.0, 'iter': 19140, 'loss': 0.3432

# Results

========================================================================================================================
Paper
----------
acc_seen: 59.6       

acc_novel: 56.7      

H: 58.1

========================================================================================================================
Original Implementation
-----------------------------------
batch_size = 50

nepoches = 20

lambda_ = 0.1

lr = 0.0001

weight_decay = 0.0001

momentum = 0.9

bias_seen 0 bias_unseen 0

iter: 2820

loss: 0.8257772922515869         loss_CE: 0.7006416320800781         loss_cal: 1.251356840133667

acc_seen: 0.5979835391044617     acc_novel: 0.5665391087532043
H: 0.5818367928121845            acc_zs: 0.6588420867919922

=======================================================================================================================

My Implementation
---------------------------
batch_size = 32

nepoches = 100

lambda_ = 0.1

lr = 0.0001

weight_decay = 0.0001

momentum = 0.9

bias_seen 0 bias_unseen 0

iter: 22000
loss: 0.448726087808609          loss_CE: 0.29331687092781067          loss_cal: 1.5540921688079834
acc_seen: 0.5924230217933655     acc_novel: 0.5670598745346069         
H: 0.5794640446589812            acc_zs: 0.6518011689186096

==========================================================================================================================

My Implementation 2
------------------------------
batch_size = 32

nepoches = 100

lambda_ = 0.1

lr = 0.0001

weight_decay = 0.0001

momentum = 0.9

bias_seen 0 bias_unseen 0

iter: 22000

loss: 0.3151090145111084            loss_CE: 0.1277414858341217           loss_cal: 1.8736752271652222
acc_seen: 0.5973256826400757        acc_novel: 0.5632122159004211
H: 0.5797675745997353               acc_zs: 0.653940737247467

==========================================================================================================================
